In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

DATA_DIR = "/content/drive/MyDrive"

os.chdir(DATA_DIR)
os.getcwd()

'/content/drive/MyDrive'

In [ ]:
%pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
df = pd.read_csv("final_dataset.csv")
df = df.dropna()
df = df.rename(columns={"type": "label"})
df

,text,source,formality,label
0,delegation peru greatly pleased sir election p...,un_debates,formal,spoken
1,webvtt kind captions language en want video tr...,khan_academy,educational,spoken
2,congratulate mr president assuming responsible...,un_debates,formal,spoken
3,well called c chosen,telephone_convos,informal,spoken
4,regards janine cash vi confirmation request ka...,enron_emails,formal,written
...,...,...,...,...
2838,hi baby im sat bloody bus mo wont home wanna s...,sms,informal,written
2839,rents havent looked rents long time,telephone_convos,informal,spoken
2840,forgetting hurts knowing decision take sometim...,medium,semiformal,written
2841,created prototypes played friends encouraged t...,medium,semiformal,written


In [ ]:
import pandas as pd

selected_columns = ["text", "label"]
df_subset = df[selected_columns]

#randomly select
df_sampled = df_subset.sample(n=500, random_state=42)

df_sampled = df_sampled.reset_index(drop=True)

print(df_sampled.head())

                                                text    label
0  madam secretary please close voting results ey...   spoken
1      ugh hopefully asus ppl dont randomly reformat  written
2  wanted wish happy new year wanted talk legal a...  written
3                          okay ive seen pick friday  written
4  like express sincere congratulations mr gurira...   spoken


In [ ]:
id2label = {0: "written", 1: "spoken"}
label2id = {"written": 0, "spoken": 1}

In [ ]:
df_sampled["label"] = df_sampled["label"].map(label2id)

In [ ]:
dataset = Dataset.from_pandas(df_sampled)
# plit into train/test sets
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [ ]:
train_dataset[0]

{'text': 'c copyright financial times ltd rights reserved', 'label': 0}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_dataset['label'][0]

0

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(train_dataset) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    eval_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
!pip install wandb

In [ ]:
! wandb login
# API KEY = 8544b034b0ba783804d4ba2d8bcdbeb3fe2de75c

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivanipuli (shivanipuli-university-of-chicago) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
import wandb
from transformers import TrainingArguments

wandb.init(project="bert-written-classification")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./hugging_face_bert_classifier",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    report_to="wandb",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    load_best_model_at_end=True,
)


ModuleNotFoundError: No module named 'wandb'

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-69-eb4c7ea1fbf5>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.247203,0.930000,0.933523,0.930000,0.929310
2,No log,0.180364,0.940000,0.943501,0.940000,0.940242
3,No log,0.160111,0.940000,0.943501,0.940000,0.940242
4,No log,0.200031,0.950000,0.950325,0.950000,0.950066
5,No log,0.296093,0.920000,0.927395,0.920000,0.920386


TrainOutput(global_step=125, training_loss=0.17392112731933593, metrics={'train_runtime': 260.7304, 'train_samples_per_second': 7.671, 'train_steps_per_second': 0.479, 'total_flos': 526222110720000.0, 'train_loss': 0.17392112731933593, 'epoch': 5.0})

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.16011103987693787, 'eval_accuracy': 0.94, 'eval_precision': 0.9435006021678041, 'eval_recall': 0.94, 'eval_f1': 0.9402424242424243, 'eval_runtime': 3.3242, 'eval_samples_per_second': 30.083, 'eval_steps_per_second': 2.106, 'epoch': 5.0}


In [ ]:
trainer.save_model("./bert_on_final_dataset")
tokenizer.save_pretrained("./bert_on_final_dataset")

('./bert_on_final_dataset/tokenizer_config.json',
 './bert_on_final_dataset/special_tokens_map.json',
 './bert_on_final_dataset/vocab.txt',
 './bert_on_final_dataset/added_tokens.json',
 './bert_on_final_dataset/tokenizer.json')

In [ ]:
trainer.save_state()